# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

# Find the ground state using direct minimisation (always using SCF is boring ...)
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.114105e+02     1.892898e+00
 * time: 0.12483596801757812
     1     1.105060e+01     1.060201e+00
 * time: 1.001235008239746
     2    -1.166195e+01     1.264238e+00
 * time: 1.1387660503387451
     3    -3.390021e+01     9.539114e-01
 * time: 1.3049030303955078
     4    -4.749239e+01     6.332177e-01
 * time: 1.4586901664733887
     5    -5.692771e+01     2.100981e-01
 * time: 1.6083781719207764
     6    -5.986171e+01     1.498246e-01
 * time: 1.7271201610565186
     7    -6.098744e+01     5.881844e-02
 * time: 1.8379809856414795
     8    -6.142003e+01     5.323389e-02
 * time: 1.9552781581878662
     9    -6.166021e+01     5.925177e-02
 * time: 2.074882984161377
    10    -6.185510e+01     3.110971e-02
 * time: 2.197175979614258
    11    -6.199988e+01     2.474754e-02
 * time: 2.3117361068725586
    12    -6.207137e+01     1.933669e-02
 * time: 2.430124044418335
    13    -6.212123e+01     1.263237e-02
 * time: 2.5492300987243

In [4]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671071
    AtomicLocal         -18.8557669
    AtomicNonlocal      14.8522632
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485376 
    Xc                  -19.3336822

    total               -62.261666462208
